##### CAMADA SILVER - ENRIQUECIMENTO #####

In [ ]:
#pip install duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import duckdb
import pandas as pd
import os
from datetime import datetime

In [5]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [32]:
df = con.execute("""
    select * from (             
    select *, row_number() over (partition by NATBR order by data_ingestao desc) as row
    from bronze_Z0019
    where data_ingestao >= '2025-11-29 00:00:00'
    ) where row = 1
                 order by NATBR asc
    """).fetch_df()
display(df)

,NATBR,MAKTX,WERKS,MAINS,LABST,arquivo,data_ingestao,row
0,10001,PARAFUSO,BT10,100,100,Z0019_01.csv,2025-11-29 23:11:50.350603,1
1,10002,MARTELOS,BT50,100,1500,Z0019_01.csv,2025-11-29 23:11:50.350603,1
2,10003,PREGO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727,1
3,10004,SERRA,BT50,100,200,Z0019_02.csv,2025-11-29 23:14:45.331727,1
4,10005,MACHADO,BT10,100,60,Z0019_02.csv,2025-11-29 23:14:45.331727,1


In [33]:
df_final = df.drop(columns=['row','data_ingestao','arquivo'])
df_final = df_final.rename(columns={'NATBR':'id'})
df_final = df_final.rename(columns={'MAKTX':'nome'})
df_final = df_final.rename(columns={'WERKS':'categoria'})
df_final = df_final.rename(columns={'MAINS':'fornecedor'})
df_final = df_final.rename(columns={'LABST':'preco'})
df_final.head()

,id,nome,categoria,fornecedor,preco
0,10001,PARAFUSO,BT10,100,100
1,10002,MARTELOS,BT50,100,1500
2,10003,PREGO,BT10,100,60
3,10004,SERRA,BT50,100,200
4,10005,MACHADO,BT10,100,60


In [34]:
#verificar os tipos de dados
df_final.dtypes

id            object
nome          object
categoria     object
fornecedor    object
preco         object
dtype: object

In [35]:
#tudo está como srting, converter para os tipos corretos
df2 = df_final
df2 = df2.astype({'id':'int', 'nome':'str', 'categoria':'str', 'fornecedor':'int', 'preco':'float'})
df2.dtypes

id              int64
nome           object
categoria      object
fornecedor      int64
preco         float64
dtype: object

In [38]:
con.execute("CREATE TABLE IF NOT EXISTS produtos (" \
    "id BIGINT, nome TEXT, categoria TEXT, fornecedor BIGINT, preco FLOAT)")

In [39]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [40]:
df_resultado = con.execute("select * from produtos").fetch_df()
df_resultado.head(10)

,id,nome,categoria,fornecedor,preco
0,10001,PARAFUSO,BT10,100,100.0
1,10002,MARTELOS,BT50,100,1500.0
2,10003,PREGO,BT10,100,60.0
3,10004,SERRA,BT50,100,200.0
4,10005,MACHADO,BT10,100,60.0


In [41]:
con.close()